# Pipeline B

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import shapiro
import seaborn as sns
from scipy.stats import zscore, skew
from sklearn import preprocessing
from itertools import chain
pd.set_option('display.max_columns', 500)

## Step 5: Standardize outliers with IQR
- Alter some of the outlier values to be at the end of the 1.5 * IQR range for more accuracy in model

In [2]:
categorical_df = pd.read_csv('categorical_df.csv').drop(columns = ['Unnamed: 0'])
continuous_df = pd.read_csv('continuous_df.csv').drop(columns = ['Unnamed: 0'])
target_df = pd.read_csv('target_df.csv').drop(columns = ['Unnamed: 0'])

In [3]:
def interquartile_range_replacement(df, col):
    '''
    Use interquartile range to address outliers and update columns showing those we augmented to reflect the changes made
    '''
    quartile_data = df[col].describe()
    q1 = quartile_data['25%']
    q3 = quartile_data['75%']
    iqr = q3 - q1
    high = quartile_data['50%'] + (1.5 * iqr)
    low = quartile_data['50%'] - (1.5 * iqr)
    
    outliers_augmented = df[col].apply(lambda x: low if x < low else (high if x > high else x))
    
    df[col + '_iqr_standardized'] = outliers_augmented
    #df['data_updated'] = np.where((df['data_updated'] == 1) | (df[col] != df[col + '_iqr_standardized']), 1, 0)
    return df

In [4]:
def min_max_scale_col(df, cols):
    scaler = preprocessing.MinMaxScaler()
    return scaler.fit_transform(new_df[cols])

def min_max_scale_df(df):
    scaler = preprocessing.MinMaxScaler()
    return scaler.fit_transform(df)

Leverage IQR to address outliers

In [5]:
cont_iqr_standardized = continuous_df.copy()
for i in [col for col in list(cont_iqr_standardized.columns[1:])]:
    cont_iqr_standardized = interquartile_range_replacement(cont_iqr_standardized, i)

# Step 6: Data Standardization

## Continuous Values
- Min-Max Scaling
- Binning

## Categorical
- Identify viable categories (create as few as possible)
- One-hot-encode if needed

In [6]:
cont_min_max_scaled = pd.DataFrame(min_max_scale_df(cont_iqr_standardized[cont_iqr_standardized.columns.tolist()[1:]]), 
             columns = cont_iqr_standardized.columns.tolist()[1:])
cont_min_max_scaled['I1'] = cont_iqr_standardized['I1'].values
cont_min_max_scaled.head()

,C7_imputed,C1,C4,C3_imputed,words_per_sentence,real_word_per,non_real_word_per,short_sentences,short_sentences_per,long_sentences_per,long_words_per,pos_words_percent,neg_words_percent,uncertain_words_percent,total_shares,sales_per_share,outstanding_share_per,offering_share_per,C5_prime,C6_prime,C7_imputed_iqr_standardized,C1_iqr_standardized,C4_iqr_standardized,C3_imputed_iqr_standardized,words_per_sentence_iqr_standardized,real_word_per_iqr_standardized,non_real_word_per_iqr_standardized,short_sentences_iqr_standardized,short_sentences_per_iqr_standardized,long_sentences_per_iqr_standardized,long_words_per_iqr_standardized,pos_words_percent_iqr_standardized,neg_words_percent_iqr_standardized,uncertain_words_percent_iqr_standardized,total_shares_iqr_standardized,sales_per_share_iqr_standardized,outstanding_share_per_iqr_standardized,offering_share_per_iqr_standardized,C5_prime_iqr_standardized,C6_prime_iqr_standardized,I1
0,0.001671,0.053924,0.749962,0.169794,0.372945,0.944727,0.055273,0.200306,0.224992,0.775008,0.580726,0.470416,0.457657,0.507764,0.017940,0.011705,0.745692,0.254308,0.035990,0.478927,0.103267,0.554455,0.646105,1.000000,0.524304,0.650533,0.349467,0.526749,0.458866,0.541134,0.586770,0.469847,0.569242,0.464101,0.523850,0.085818,0.521728,0.478272,0.522171,0.684926,AATI
1,0.000843,0.119884,0.583748,0.168708,0.394163,0.934175,0.065825,0.371560,0.219755,0.780245,0.550219,0.314935,0.601349,0.495247,0.009994,0.009744,0.913135,0.086865,0.118042,0.348659,0.052090,1.000000,0.335391,0.054639,0.564129,0.521883,0.478117,0.987654,0.443639,0.556361,0.480447,0.158061,0.813082,0.441796,0.291806,0.071443,1.000000,0.000000,1.000000,0.380833,ABPI
2,0.000238,0.038517,0.717213,0.168790,0.334588,0.945220,0.054780,0.053517,0.229360,0.770640,0.661030,0.945124,0.151358,0.411846,0.006303,0.003949,0.715407,0.284593,0.031013,0.143236,0.014711,0.396040,0.584885,0.159107,0.452311,0.656534,0.343466,0.131687,0.471565,0.528435,0.866645,1.000000,0.049463,0.293168,0.184050,0.028953,0.434328,0.565672,0.439812,0.000000,ACAD
3,0.000275,0.095811,0.714502,0.168861,0.291162,0.953234,0.046766,0.172783,0.346964,0.653036,0.659600,0.768541,0.196538,0.517175,0.005325,0.005162,0.710602,0.289398,0.030315,0.277011,0.017024,0.985149,0.579818,0.249828,0.370805,0.754237,0.245763,0.452675,0.813476,0.186524,0.861660,1.000000,0.126132,0.480873,0.155497,0.037847,0.420464,0.579536,0.428264,0.213581,ACHN
4,0.020605,0.033702,0.499347,0.169072,0.323613,0.923515,0.076485,0.302752,0.289146,0.710854,0.519817,0.423133,0.499742,0.505592,0.013018,0.191097,0.737886,0.262114,0.034602,0.443103,1.000000,0.346535,0.177617,0.519244,0.431712,0.391930,0.608070,0.802469,0.645379,0.354621,0.374488,0.375031,0.640658,0.460230,0.380111,1.000000,0.499201,0.500799,0.499207,0.601301,ACLI


In [8]:
def combine_industry_col(val):
    if val in ['Mining','Finance, Insurance and Real Estate', 'Construction', 'Agriculture, Forestry and Fishing']:
        return 'Other'
    elif val == 'Mining':
        return 'Other'
    elif val in ['Retail Trade','Wholesale Trade', 'Transportation, Communications, Electric, Gas and Sanitary service']:
        return 'Other'
    else:
        return val

In [9]:
categorical_df['industry_bin'] = categorical_df['industry_bin'].apply(combine_industry_col)

In [10]:
def one_hot_encode(df, col):
    return pd.get_dummies(df[col], prefix = col + '_')

In [11]:
categorical_df = pd.concat([categorical_df, one_hot_encode(categorical_df, 'industry_bin')], axis = 1)

In [12]:
categorical_df.drop(columns=['industry_bin'], inplace = True)

In [13]:
cont_min_max_scaled.to_csv('cont_min_max_scaled_standardized.csv')
categorical_df.to_csv('categorical_df_standardized.csv')

In [14]:
# cont_min_max_scaled = pd.read_csv('cont_min_max_scaled_standardized.csv')
# categorical_df = pd.read_csv('categorical_df_standardized.csv')

In [15]:
#target_df = pd.read_csv('target_df.csv')